In [ ]:
IJulia.SOFTSCOPE[] = false

# Add current directory to load path, to allow loading modules like GPFields.
if "." ∉ LOAD_PATH
    push!(LOAD_PATH, ".")
end
@show Base.load_path()

using Circulation
using GPFields

In [ ]:
const DATA_DIR_BASE = expanduser("~/Dropbox/circulation/data/4vortices")
const DATA_IDX = 1

const GP_PARAMS = ParamsGP(
    (256, 256),
    c = 1.0,
    nxi = 1.5,
)

In [ ]:
# Load data
psi = let datadir = joinpath(DATA_DIR_BASE, string(GP_PARAMS.dims[1]))
    @show datadir
    @time GPFields.load_psi(GP_PARAMS, datadir, DATA_IDX)
end
summary(psi)

In [ ]:
rho = GPFields.compute_density(psi)
summary(rho)

In [ ]:
p = GPFields.compute_momentum(psi, GP_PARAMS);
v = map(pj -> pj ./ (rho .+ 1e-12), p);
vreg = map(pj -> pj ./ sqrt.(rho), p);
@show extrema(p[1]);
@show extrema(v[1]);
@show extrema(vreg[1]);

In [ ]:
using FFTW

# FFT of v[i] along dimension i
vf = ntuple(i -> rfft(v[i], i), 2);

# Wave numbers for r2c transforms
Ns = GP_PARAMS.dims
Ls = GP_PARAMS.L
fs = 2pi .* Ns ./ Ls  # sampling frequency = 2pi * N / L
ks = rfftfreq.(Ns, fs);

# Make loop around (Nx/4, Ny/4)
I0 = Ns .>> 2  # N / 2^2 = N / 4
dI = Ns .>> 6  # N / 2^6 = N / 64
loop = Circulation.Rectangle(I0 .- dI, 2 .* dI)

# In physical coordinates
loop_phys = (Ls ./ Ns) * loop

println(loop)
println(loop_phys)

@btime circulation($loop, $vf, $ks, $Ns)
# @code_warntype circulation(loop, vf, ks, Ns)
circulation(loop, vf, ks, Ns)

# Plots

In [ ]:
@time import PyPlot
const plt = PyPlot;

In [ ]:
fig, ax = plt.subplots()
x, y = get_coordinates(GP_PARAMS)
ax.set_aspect(:equal)
cf = ax.contourf(x, y, rho')
# cf = ax.contourf(x, y, v[1])
fig.colorbar(cf)
ax.quiver(x, y, p[1]', p[2]', scale=12)
# ax.quiver(x, y, v[2]', v[1]', scale=15)
ax.set_xlim(0.4pi, 0.6pi)
ax.set_ylim(0.4pi, 0.6pi)

# Test integration

In [ ]:
let L = 4pi
    N = 256
    x = range(0, L, length=N + 1)[1:end-1]
    
    v = @. 0.2 + sin(x) + cos(2x)
    vI = @. 0.2x - cos(x) + sin(2x) / 2
    
    vf = rfft(v)
    k = rfftfreq(N, 2pi * N / L)
    @assert k[2] ≈ 2pi / L

    a = searchsortedlast(x, 0)
    b = searchsortedlast(x, π / 4)
    
    @show a, b
    @show x[[a, b]]
    int1 = vI[b] - vI[a]
    int2 = Circulation.integrate(vf, k, x[a], x[b]) / N
    @show int1
    @show int2
    @assert int1 ≈ int2
    @btime Circulation.integrate($vf, $k, 0.1, 0.4)
end

# VTK files (for verification)

In [ ]:
using WriteVTK

@show typeof(x)

vtk_grid("fields", x, y, compress=false) do vtk
    vtk["rho"] = rho
    vtk["v"] = v
    vtk["p"] = p
end